<a href="https://colab.research.google.com/github/Bast-94/ML-BIO/blob/exercice-12/exercice_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice 12:

[lien Nowledgeable](https://nowledgeable.com/invitation/student/join-module/9c8ec467-686a-44cd-a2f2-85cf174a79ad)
1. Tester un modèle de convolution 1D sans faire des extractions de features.

2. Tester un classifier avec et sans faire la transformée en ondelettes pour savoir si le modèle donne une meilleur résultat avec la transformée en ondelettes ?

3. Tester des modèles de classification différentes  comme RadomForest, ExtraTrees, GradientBoosting, etc.

4. Tester des différentes familles d'ondelettes comme Symlets, Coiflets, Biorthogonal etc.

5. Tester d'autres features (vous pouvez vous référer des listes de features avec des librairies tsfresh ou tsfel )

6. A l'aide de la transformée en ondelettes continue, transformez des signaux ECG en scaleograms et puis utilisez un modèle CNN pour classifier des images.


In [ ]:
!pip install wfdb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)

import wfdb
import glob
import os
import pywt
import tensorflow as tf
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Dropout, Conv2D, Input, MaxPooling2D,Flatten

In [ ]:
def load_signal_using_wfdb(file, start, end, channel, pn_dir):
    """
    file: name of record
    start: int
    end:int
    channel: 0 or 1
    """
    record = wfdb.rdrecord(file, sampfrom = start, sampto = end, channels=[channel], pn_dir=pn_dir)
    data = record.p_signal.reshape(-1)
    return data


In [ ]:
database_names = ['nsrdb','mitdb','chfdb']
n_label = len(database_names)

In [ ]:
record_names = []
labels = []

for i,name in enumerate(database_names):
    current_record_names = wfdb.get_record_list(name)
    record_names += current_record_names
    labels += [i for record_name in current_record_names]
assert len(record_names) == len(labels)

In [ ]:
X_cwt = []
sample_size = 128
scale_size = 64
scales = range(1, scale_size+1)
waveletname = 'morl'
for record_name,label in tqdm(zip(record_names,labels)):
    
    signal=(load_signal_using_wfdb(record_name, start=0, end=sample_size, channel = 0, pn_dir=database_names[label]))
    coeff, freqs = pywt.cwt(signal, scales, waveletname, 1)
    X_cwt.append(coeff)
X_cwt = np.array(X_cwt)

In [ ]:
labels = np.array(labels)
num_classes = len(np.unique(labels))
y_train = np.eye(num_classes)[labels]

In [ ]:
assert X_cwt.shape[0]  == y_train.shape[0]

In [ ]:
X_cwt = np.reshape(X_cwt,(X_cwt.shape[0],X_cwt.shape[1],X_cwt.shape[2],1))
X_cwt.shape

In [ ]:
def build_model(input_shape,num_class):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(filters=128,kernel_size=5))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=64,kernel_size=5))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_class,activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model
model = build_model(X_cwt.shape[1:],y_train.shape[1])
model.summary()

In [ ]:
model.fit(X_cwt,y_train,validation_split=0.2,epochs=20)

In [ ]:
signal_test = load_signal_using_wfdb('208', start=0, end=2000, channel = 0, pn_dir='mitdb')
waveletname = 'morl'
scales = range(1, 128)
coeff, freqs = pywt.cwt(signal_test, scales, waveletname, 1)
print(coeff.shape)
N = signal_test.size
sr = 360
dt = 1/sr
time = np.arange(0, N)*dt
fig, (ax1, ax2) = plt.subplots(2, figsize=(10, 5))
ax1.plot(time, signal_test)
im = ax2.imshow(abs(coeff[:,:]), extent = [0, 200, 30, 1],
                interpolation = 'bilinear', aspect = 'auto')